# Roman ETC Tools

This notebook is to demonstrate options for streamlining the use of the exposure-time calculator (ETC) written for the *Nancy Grace Roman Space Telescope* (Roman) called ```pandeia``` (see [pandeia homepage](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial)).  The primary code ```pandeia.engine``` primarly takes two inputs: (1) an instrumental set up and (2) a scene of astronomical sources and produces an output dictionary (see [pandeia reports](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-reports)). Importantly, the output dictionary contains the best estimate for several key predictions: the signal-to-noise (S/N), exposure time, and expected flux (in e-/s).  However, some users may find that computing S/N is not quite what is needed, but rather would prefer to tweak the properties of the instrumental set up and/or astronomical scene for a specified S/N.



**Author:** Russell Ryan  
**Date:** May 23, 2022  
**Editor:** Sebastian Gomez, Tyler Desjardins, Andreea Petric

This notebook was written with these key packages and version numbers:

| package        | version |
| -------------- | ------- |
| pandeia.engine | 1.7.1   |
| scipy          | 1.7.1   |
| numpy          | 1.21.1  |



## Primary Functions

As eluded to above, there are three relevant "quantities" in an ETC (something that emulates exposure time, source brightness, and signal-to-noise), and the point of this suite is to compute any one of these quantities, given the other two.  As such, there are primarily three functions that users will likely use:

1. ```soc_roman_tools.utilities.pandeia_notebook.compute_sn```
2. ```soc_roman_tools.utilities.pandeia_notebook.compute_mag```
3. ```soc_roman_tools.utilities.pandeia_notebook.compute_nexp```

**An Aside on Roman/WFI Exposure Times:** The Roman/WFI employs a non-destrcutive, up-the-ramp sampling method, wherein the exposure time is determined from the combination of the readout pattern and number of groups and exposures.  For the purposes of this notebook, we hold the number of groups defaulted as ```ngroup```=10, therefore the number of exposures ```nexp``` is the tunable parameter that govens exposure time.  





## Function Inputs

The inputs to every function are largely the same, parameter(s) governing the instrumental setup and astronomical sources.  Full details of these functions are included in the ```soc_roman_tools.utilities.pandeia_notebook.py``` file, but in brief they are:

### Instrumental Setup

- **background:** low sky background, see [Pandeia Backgrounds](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-backgrounds) for more information on the background models and fiducial levels.
- **measurement aperture:** $r=0.2''$ with sky annulus of $0.6''<r<0.8''$
- **detector readout:** the current expectation for running the Roman/WFI is that there will be a single integration (hence ```nint```=1), and the user is free to alter the value of ```ngroup``` (but is defaulted to 10).  However, one should not reduce ```ngroup``` below ~4 as the assumptions on the first-frame property and readnoise become dominant.  For more information on the detector readouts, please see the [Understanding Exposure Times](https://jwst-docs.stsci.edu/understanding-exposure-times) article in the JWST documentation.

Therefore, the key tunable parameter is taken to be ```nexp```, which must always be an integer.


### Astronomical Scene

- **spatial morphology:** point source
- **spectral morphology:** $f_\nu = $ constant
- **magnitude system:** AB 

With that, there is only one tunable parameter: the normalization constant of the spectrum, which is set as an AB magnitude normalization.  Therefore, the wavelength and "type" of normalization does not matter, but are defaulted as 2.0 $\mu$m and ```at_lambda```, respectively.  Therefore, the AB magnitude as ```norm_flux``` is the parameter that governs the astronomical scene.


## Function Outputs

Each of the primary three functions returns two quantities: (1) the quantity of interest and (2) the output dictionary from ```pandeia.engine``` (the details of this dictionary are given in the [pandeia reports](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-reports)).  This notebook will not delve into this dictionary, but rather use only the quantities of interest.


In [ ]:
# load the important modules
import os
from soc_roman_tools.utilities import pandeia_notebook

# print the version of the reference data and code base:

print('path to reference files: '+os.environ['pandeia_refdata'])
for k,v in pandeia_notebook.pandeia_version().items():
    print(f'{k:>23}:',v)

## Set the Filter
This notebook works for the imaging modes, and so the options for the filter are: ```f062```, ```f087```, ```f106```, ```f129```, ```f146```, ```f158```, ```f184```, ```f213``` 



In [ ]:
# show the valid filters:
print(pandeia_notebook.VALID_FILTERS)

# set the global variable for the filter name (change to any valid filter)
FILTER = 'f129'

## Option 1: Compute S/N

This is largely the standard way of running ```pandeia```, where the properties of the instrumental set up and astronomical scene are specified.  ***This step may generate a WARNING from synphot that the spectrum is extrapolated, which can be safely ignored.***


In [ ]:
mag = 25.0    # assumed magnitude
nexp = 3      # number of exposures
sn, etc = pandeia_notebook.compute_sn(FILTER, mag, nexp)
print(f'Estimated S/N: {sn:.2f}')

## Option 2: Compute magnitude
In this example, we assume one has a required signal-to-noise and a fixed number of exposures (hence exposure time), and is interested in knowing what magnitude limit they can achieve with this setup.

In [ ]:
sn = 5.     # required S/N
nexp = 10   # number of exposures to simulate
mag, etc = pandeia_notebook.compute_mag(FILTER, sn, nexp)
print(f'Estimated magnitude: {mag:.2f}')

## Option 3: Compute Nexp
In this example, we assume one has a required signal-to-noise and desired magnitude limit, and wishes to know the number of exposures required to achieve this setup. ***This step may generate a WARNING from synphot that the spectrum is extrapolated, which can be safely ignored. There may be an additional WARNING that the S/N for a single exposure is larger than what was requested, which can be ignored.***

*Nota bene: Since nexp must be an integer, the actual S/N will be at least the required value. This is most pronounced for when the infered nexp is small (ie. bright sources and/or very high S/N).  These effects will be demonstrated.*

In [ ]:
# first consider a bright source with high S/N
mag = 24.
sn = 20.
nexp, etc = pandeia_notebook.compute_nexp(FILTER, sn, mag)

# Given this setup, only 1 exposure will be needed.  But looking in the etc report, we see that the 
# actual S/N achieved by this exceeds the required amount.
print(f'number of exposures: {nexp}')
print(f'actual S/N reached: {etc["scalar"]["sn"]:.2f}')

In [ ]:
# do it again, but now with a setting that will require multiple nexp
# this will now be much slower than the previous, as it requires iteration.  the pervious example
# short circuits the calculation by first testing if >1 exposure is needed.
mag = 28.
sn = 6.
nexp, etc = pandeia_notebook.compute_nexp(FILTER, sn, mag)
print(f'number of exposures: {nexp}')
print(f'actual S/N reached: {etc["scalar"]["sn"]:.2f}')

## Other Examples



### Round-Trip Example
In the preceeding, we showed how there are (essentially) three relevant quantities in the ETC (magnitude, signal-to-noise, and number of exposures), and given any two of these the third can be inferred.  Here we show that these calculations can be done in a *round-trip* fashion, so that the package is self consistent.

In [ ]:
# first assume option 1 and compute signal-to-noise
mag0 = 24.
nexp0 = 10
sn, etc = pandeia_notebook.compute_sn(FILTER,mag0,nexp0)

# now take that S/N and nexp to compute the magnitude, which should be equal to mag0 (by construction)
mag1, etc = pandeia_notebook.compute_mag(FILTER,sn,nexp0)

# final step, take this magnitude and previously inferred S/N and compute number of exposures, which again, 
# should be equal to the nexp0 (by construction)
nexp1, etc = pandeia_notebook.compute_nexp(FILTER,sn,mag1)

print(f'Input magnitude: {mag0:.2f}')
print(f'Inferred magnitude: {mag1:.2f}')
print(f'Input nexp: {nexp0}')
print(f'Inferred nexp: {nexp1}')

### Change Defaults to Pandeia

Above, we assumed a default setting in the ```pandeia_notebook.DEFAULTS```, but these can be set as optional arguments to any of the three primary functions in ```pandeia_notebook```.

In [ ]:
# show the defaults:
for k,v in pandeia_notebook.DEFAULTS.items():
    print(f'{k:>28}: {v}')

In [ ]:
# let's change one of the inputs to see how Pandeia reacts
mag = 25.0    # assumed magnitude
nexp = 3      # number of exposures
sn_def, etc = pandeia_notebook.compute_sn(FILTER, mag, nexp)
sn_new, etc = pandeia_notebook.compute_sn(FILTER, mag, nexp,ngroup=4)

# now the new S/N should be a lesser value than the original, as reducing the number of groups leads to 
# less exposure time and hence lower S/N
print(f'Default ngroup S/N: {sn_def:.2f}')
print(f'Reduced ngroup S/N: {sn_new:.2f}')

In [ ]:
# perhaps another change the aperture
sn_new, etc = pandeia_notebook.compute_sn(FILTER, mag, nexp,aperture_size=0.4)
print(f'Larger source aperture S/N: {sn_new:.2f}')

# perhaps increase the sky background
sn_new, etc = pandeia_notebook.compute_sn(FILTER, mag, nexp,background_level='high')
print(f'Increased sky background S/N: {sn_new:.2f}')